In [6]:
import numpy as np
import pandas as pd
from pandas_datareader import DataReader as pdr
import yfinance as yf
import plotly.graph_objects as go

# U.S. STOCK MARKET RETURN

In [7]:
mkt = pdr('F-F_Research_Data_Factors','famafrench',start=1926)[1]/100
mkt = mkt['Mkt-RF'] + mkt.RF
mkt.index.name = "year"
mkt.name = "market"
mkt.index = mkt.index.astype(str)
mkt.tail()

year
2018   -0.0512
2019    0.3043
2020    0.2411
2021    0.2360
2022   -0.2017
Name: market, dtype: float64

# RETURNS FROM YAHOO FINANCE

In [8]:
ticker = input("Input a ticker")
price = yf.download(ticker, start="1970-01-01")["Adj Close"]
price = price.resample("Y").last()
ser = price.pct_change()[:-1].dropna()
ser.index = ser.index.map(lambda x: x.year)
ser.index.name = "year"
ser.name = ticker
ser.tail()

[*********************100%***********************]  1 of 1 completed


year
2018   -0.097474
2019    0.152737
2020   -0.259521
2021    0.462413
2022    0.584623
Name: cvx, dtype: float64

# YAHOO RETURN OR MARKET

In [9]:
flag = int(input("input 0 for market or 1 for yahoo return"))
data = ser if flag else mkt
name = f"{ticker.upper()} Return" if flag else "Market Return"

# FIGURE 1

In [10]:
trace = go.Scatter(
    x=data.index.to_list(),
    y=data, 
    mode="lines", 
    hovertemplate="%{x}<br>%{y:.1%}"
)
fig = go.Figure(trace)
fig.update_layout(
    xaxis_title="Year",
    yaxis_title=name,
    yaxis_tickformat=".0%",
    template="plotly_white",  
)
fig.show()

# FIGURE 2

In [19]:
mean = data.mean()
T = len(data)
geomean = (1+data).prod()**(1/T) - 1

trace1 = go.Scatter(
    x=data.index.to_list(),
    y=(1+data).cumprod(),
    mode="lines",
    name="Actual Accumulation",
    hovertemplate="%{y:.2f}"
)
trace2 = go.Scatter(
    x=data.index.to_list(),
    y=(1+mean)**np.arange(1, T+1),
    mode="lines",
    name="Compounded Arithmetic Avg",
    hovertemplate="%{y:.2f}"
)
trace3 = go.Scatter(
    x=data.index.to_list(),
    y=(1+geomean)**np.arange(1, T+1),
    mode="lines",
    name="Compounded Geometric Avg",
    hovertemplate="%{y:.2f}"
)

fig = go.Figure()
for trace in (trace1, trace2, trace3):
    fig.add_trace(trace)
fig.update_layout(
    xaxis_title="Year",
    hovermode="x unified",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
    ),
    template="plotly_white"
)
fig.show()